Testing FIBSEM Microscope overall class
    - Testing Thermo Fisher instance of class
    

Importing Relevant Modules

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire
import fibsem.alignment as fa
from fibsem.structures import BeamType, ImageSettings, GammaSettings, FibsemRectangle, FibsemImage, check_data_format
import matplotlib.pyplot as plt
from fibsem import FibsemMicroscope, calibration
import os
import logging
from fibsem.utils import current_timestamp, configure_logging
from pathlib import Path
from autoscript_sdb_microscope_client.structures import (
    AdornedImage,
    GrabFrameSettings,
    Rectangle,
    RunAutoCbSettings,
)

Importing settings from config

In [ ]:
settings = utils.load_settings_from_config()


Testing Setup 

In [ ]:
microscope, settings = utils.setup_session()

Setting up for image acquisition

In [ ]:
print(settings)

In [ ]:
# set gamma settings
gamma_settings = GammaSettings(
        enabled=True,
        min_gamma=0.5,
        max_gamma=1.8,
        scale_factor=0.01,
        threshold=46,
    )

# set imaging settings
image_settings = ImageSettings(
        resolution="1536x1024",
        dwell_time=1.0e-6,
        hfw=150.0e-6,
        autocontrast=True,
        beam_type=BeamType.ELECTRON,
        gamma=gamma_settings,
        save=True,
        save_path="fibsem\\test_images",
        label=utils.current_timestamp(),
        reduced_area=None,
    )

frame_settings = GrabFrameSettings(
    resolution=image_settings.resolution,
    dwell_time=image_settings.dwell_time,
    reduced_area=image_settings.reduced_area,
)

 # set label
if image_settings.beam_type is BeamType.ELECTRON:
    label = f"{image_settings.label}_eb"

if image_settings.beam_type is BeamType.ION:
    label = f"{image_settings.label}_ib"

Autoconcrast

In [ ]:
microscope.autocontrast(beam_type=image_settings.beam_type)
microscope.autocontrast(beam_type=BeamType.ION)


Take new image

In [ ]:
new_image = microscope.acquire_image(frame_settings, image_settings)
# apply gamma correction
if image_settings.gamma.enabled:
    image = acquire.auto_gamma(new_image, image_settings.gamma)

plt.imshow(new_image.data)
plt.show()

# save image
if image_settings.save:
    filename = os.path.join(image_settings.save_path, label)
    image.save(save_path=filename)
    print("Image saved")

Checks

In [ ]:
assert isinstance(new_image, FibsemImage)
check_data_format(new_image.data)

assert new_image.metadata.compare_image_settings(image_settings)
assert new_image.metadata.version == 'v1'
assert os.path.exists(f"{filename}.tif")

print(new_image.metadata.microscope_state)

Take last image

In [ ]:
label = f"{image_settings.label}_last"

last_image = microscope.last_image(image_settings.beam_type)

plt.imshow(last_image.data)
plt.show()

if image_settings.save:
    filename = os.path.join(image_settings.save_path, label)
    image.save(save_path=filename)


Checks

In [ ]:
assert isinstance(last_image, FibsemImage)
check_data_format(last_image.data)

assert last_image.metadata.compare_image_settings(image_settings)
assert last_image.metadata.version == 'v1'
assert os.path.exists(f"{filename}.tif")

print(last_image.metadata.microscope_state)

Beam shift

In [ ]:
print("electron: ", microscope.connection.beams.electron_beam.beam_shift.value)
print("ion: ", microscope.connection.beams.ion_beam.beam_shift.value)

microscope.reset_beam_shifts()

print("electron: ", microscope.connection.beams.electron_beam.beam_shift.value)
print("ion: ", microscope.connection.beams.ion_beam.beam_shift.value)

assert microscope.connection.beams.electron_beam.beam_shift.value.x == 0
assert microscope.connection.beams.electron_beam.beam_shift.value.y ==  0

assert microscope.connection.beams.ion_beam.beam_shift.value.x == 0
assert microscope.connection.beams.ion_beam.beam_shift.value.y ==  0

Disconnect from Microscope

In [ ]:
microscope.disconnect()